## load smartSLURM

In [ ]:
from py4j.java_gateway import JavaGateway
GATEWAY = JavaGateway.launch_gateway(classpath='include/smartSLURM.jar')
ServerSSH = GATEWAY.jvm.com.chanzy.ServerSSH
ServerSLURM = GATEWAY.jvm.com.chanzy.ServerSLURM

## set login parameters

In [ ]:
SSH_USER = 'username'
SSH_IP = 'hostname'
SSH_PASSWORD = 'password'
SERVER_PROJECT_PATH = '~/test-smartSLURM'

## SSH PART

In [ ]:
import os
import time
# load smartSLURM
ssh = ServerSSH.get(os.getcwd(), SERVER_PROJECT_PATH, SSH_USER, SSH_IP, SSH_PASSWORD)
# put all of this project dir to the server (will exclude file and dir which begin with '.' or '_')
ssh.putWorkingDir()
# can mkdir in ssh
ssh.mkdir('.test1/.test2/.test3')
# submit command to remote server by using 'system' like matlab
ssh.system('ls')
ssh.system('echo "I am using smartSlURM system" > .test1/.test2/.test3/out.txt')
# get dir from remote server by using 'getDir'
ssh.getDir('.test1')
ssh.getDir('.test1', 4) # using 4 threads
# set compression level for large file transmission
ssh.setCompressionLevel(4)
ssh.putDir('.test1')
# unset compression by setting a negative level
ssh.setCompressionLevel(-1)
# you can get the pool to submit commands parallel
pool = ssh.pool(4)
pool.submitSystem('sleep 1s')
pool.submitSystem('sleep 2s')
pool.submitSystem('sleep 3s')
pool.submitSystem('sleep 4s')
pool.submitSystem('sleep 5s')
start = time.time()
pool.waitUntilDone()
end = time.time()
print(end-start)
pool.shutdown()
# use 'shutdown()' to disconnect at last
ssh.shutdown()

## SLURM PART

In [ ]:
import os
# init slurm, max job number, local project path and remote project path is needed
slurm = ServerSLURM.get(4, os.getcwd(), SERVER_PROJECT_PATH, SSH_USER, SSH_IP, SSH_PASSWORD)
# you can get the ssh of this and use the ssh methods before
ssh = slurm.ssh()
# use 'submitSystem' to submit command in sbatch mod
# slurm.submitSystem(Command, [Partition='work'], NodeNumber=1, OutputPath='.temp/slurm/out-%j');
slurm.submitSystem('echo "I am using smartSlURM submitSystem"')
# use 'submitSrun' to submit a srun command directly (also in sbatch mod)
# slurm.submitSrun(Command, [Partition='work'], TaskNumber=1, MaxTaskNumberPerNode=20, OutputPath='.temp/slurm/out-%j');
slurm.submitSrun('echo "I am using smartSlURM submitSrun"')
# use 'submitSrunBash' to submit a bash file and use srun to exec (also in sbatch mod)
# slurm.submitSrunBash(BashPath, [Partition='work'], TaskNumber=1, MaxTaskNumberPerNode=20, OutputPath='.temp/slurm/out-%j');
slurm.submitSrun('path/to/your/script')
# you can use 'undo' to poll the last command in queue list
cmd = slurm.undo()
# use 'getActiveCount' and 'getQueueSize' to monitor task situation
activeNum = slurm.getActiveCount()
queueNum = slurm.getQueueSize()
taskNum = slurm.getTaskNumber() # getActiveCount() + getQueueSize();
# you can use `waitUntilDone` to wait all task complete
slurm.waitUntilDone()
# or can use the classic way in java thread pool
slurm.shutdown()
slurm.awaitTermination()

## unload smartSLURM

In [ ]:
GATEWAY.shutdown()